<a href="https://colab.research.google.com/github/amilyk/Agentic-ai-notes/blob/main/CodeAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tinydb

In [51]:
!touch .env
!echo "DEEPSEEK_API_KEY=your-deepseek-api-key" > .env
!echo "OPENAI_API_KEY=your-openai-api-key" >> .env

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [ ]:
# !pip install anthropic

In [ ]:
# ==== 导入 ====
from __future__ import annotations
import json

from openai import OpenAI
import re, io, sys, traceback, json
from typing import Any, Dict, Optional
from tinydb import Query, where

# 工具模块
import utils      # 用于提示/打印的助手函数
import inv_utils  # 用于库存、交易、模式构建和 TinyDB 数据填充的函数


client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),   # 从环境变量读取
    base_url="https://api.deepseek.com"
)

# 创建轻量表（tinydb）

In [35]:
#库存表、交易表
db, inventory_tbl, transactions_tbl = inv_utils.seed_db()

In [36]:
utils.print_html(json.dumps(inventory_tbl.all(), indent=2), title="库存表")
utils.print_html(json.dumps(transactions_tbl.all(), indent=2), title="交易表")

# LLM用代码执行计划

In [37]:
PROMPT = """你是一名高级数据助手。通过编写 TINDYDB PYTHON 代码来制定计划。

数据库模式和示例 (只读):
{schema_block}

执行环境 (已导入/已提供):
- 变量: db, inventory_tbl, transactions_tbl  # TinyDB 表对象
- 助手函数: get_current_balance(tbl) -> float, next_transaction_id(tbl, prefix="TXN") -> str
- 自然语言: user_request: str  # 原始用户消息

规划规则 (关键):
- 从 user_request 派生所有过滤器/参数 (形状/关键词, 价格范围 "低于/高于/之间", 库存提及, 数量, 购买/退货意图)。不要硬编码值。
- 使用 Query() 动态构建 TinyDB 查询。如果 user_request 中没有某个约束，则不要应用它。
- 保持保守：如果意图不明确，请执行只读操作 (演习 - DRY RUN)。

交易政策 (硬性规定):
- 不要创建聚合的多项目交易。
- 如果请求包含多个项目，则为每个项目创建单独的交易行。
- 对于每个项目：
  - 计算其自己的单行总计 (unit_price * qty),
  - 插入一笔该金额的交易,
  - 按顺序更新余额 (balance += line_total),
  - 更新该项目的库存。
- 如果任何请求的项目库存不足，不要更改任何数据；回复 STATUS="insufficient_stock"。

人类可读响应要求 (硬性规定):
- 你必须设置一个名为 `answer_text` (str 类型) 的变量，内容为简短、客户友好的句子 (1-2 行)。
- 这个句子是唯一面向用户的消息。没有数据帧/JSON，没有样板式的免责声明。
- 如果没有匹配项，礼貌地说明情况，并提供一个相近的替代方案 (最接近的款式/价格) 或下一步建议。

行动政策:
- 如果请求明确要求改变状态 (购买/采购/退货/补货/调整):
    ACTION="mutate"; SHOULD_MUTATE=True; 执行更改并写入匹配的交易行。
  否则:
    ACTION="read"; SHOULD_MUTATE=False; 模拟并作为演习 (dry run) 简要说明 (仅在日志中)。

失败与边缘情况处理 (必须实现):
- 不要在 Query.test 中捕获外部变量。将它们作为显式参数传递。
- 始终设置一个简短的 `answer_text`。同时设置一个字符串 `STATUS`，其值为以下之一:
  "success", "no_match", "insufficient_stock", "invalid_request", "unsupported_intent"。
- no_match: 没有项目满足过滤器 → 建议风格/价格最接近的，或邀请客户提供不同范围。
- insufficient_stock: 找到项目但库存 < 请求数量 → 说明可用数量，并提供你能满足的最大数量。
- invalid_request: 无法解析基本信息 (例如 购买/退货 的数量) → 简洁地询问缺失的部分。
- unsupported_intent: 该行动超出了商店的能力范围 → 提供最接近的支持替代方案。
- 在所有情况下，保持乐于助人且简洁的语气 (1-2 句话)。仅在 stdout 日志中放入技术细节 (例如 ACTION/DRY RUN)。

输出契约:
- 仅在这些标签之间返回可执行的 Python (没有额外文本):
  <execute_python>
  # 你的 python
  </execute_python>

代码核对清单 (在代码中遵循):
1) 从 user_request 解析意图和约束 (可用正则表达式)。
2) 渐进式构建 TinyDB 条件；查询 inventory_tbl。
3) 如果是变更(mutate)操作: 验证库存, 更新库存, 插入一笔交易 (新 id, 金额, 余额, 时间戳)。
4) 始终设置:
   - `answer_text` (人类可读的句子, 必需),
   - `STATUS` (参见上面的列表)。
   同时向 stdout 打印一个简短的日志, 例如 "LOG: ACTION=read DRY_RUN=True STATUS=no_match"。
5) 可选: 如果有用的话设置 `answer_rows` 或 `answer_json`，但 `answer_text` 是强制性的。

语气示例 (用于 `answer_text`):
- 成功: "是的，我们有经典款太阳镜，这是一款圆形镜框，售价60美元。"
- 无匹配: "我们目前没有100美元以下的圆形镜框库存，但我们的 Moon圆形镜框有货，售价120美元。"
- 库存不足: "Classic款我们只剩下1副了；我可以为您预留。"
- 无效请求: "我可以处理这个——请问您想购买多少副？"
- 不支持的意图: "我们不能翻新镜框，但我可以推荐类似的新款式。"

约束条件:
- 使用 TinyDB Query 进行过滤。仅在需要时使用标准库导入。
- 保持代码清晰，并使用编号步骤进行注释。

用户请求:
{question}
"""

In [38]:
# ---------- 1) 代码生成 ----------
def generate_llm_code(
    prompt: str,
    *,
    inventory_tbl,
    transactions_tbl,
    model: str = "gpt-4.1-mini",
    temperature: float = 0.2,
) -> str:
    """
    请求 LLM 生成一个“带代码的计划”响应。
    返回完整的助手内容 (包括周围的文本和标签)。
    实际的代码提取稍后在 execute_generated_code 中进行。
    """
    schema_block = inv_utils.build_schema_block(inventory_tbl, transactions_tbl)
    prompt = PROMPT.format(schema_block=schema_block, question=prompt)

    resp = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {
                "role": "system",
                "content": "你编写安全、注释良好的 TinyDB 代码来处理数据问题和更新。"
            },
            {"role": "user", "content": prompt},
        ],
    )
    content = resp.choices[0].message.content or ""

    return content

## 人工查询下库存表

In [39]:
#查找库存表中描述或名字有"圆形太阳镜"商品详情

Item = Query()                  # 创建一个 Query 对象来引用字段 (例如 Item.name, Item.description)

# 搜索库存表，查找描述(description)或名称(name)
# 包含单词 "round" (不区分大小写) 的文档。检查是内联完成的：
# - (v or "") 确保我们通过将 None 转换为空字符串来处理它
# - .lower() 规范化大小写
# - " round " 强制执行一个粗略的单词边界 (不会匹配 "wraparound")
round_sunglasses = inventory_tbl.search(
    (Item.description.test(lambda v: " round " in ((v or "").lower()))) |
    (Item.name.test(        lambda v: " round " in ((v or "").lower())))
)

# 在 notebook UI 中将结果渲染为格式化的 JSON
utils.print_html(json.dumps(round_sunglasses, indent=2), title="Inventory Status: Round Sunglasses")


    "item_id": "SG005",
    "name": "Classic",
    "description": "Classic round profile with minimalist metal frames, offering a timeless and versatile style that fits both casual and formal wear.",
    "quantity_in_stock": 10,
    "price": 60
  找到一款符合条件且价格低于100的商品（商品确实存在）


In [40]:
# 讲座中的 Andrew 示例提示
prompt_round = "你们库存里是否有名字或者描述叫圆形太阳镜的SunGlasses，且价格低于 100 美元？"

# 生成以代码为计划的响应（完整内容；可能包含 <execute_python> 标签）
full_content_round = generate_llm_code(
    prompt_round,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    model="deepseek-chat",
    temperature=1.0,
)

# 查看 LLM 的计划与代码（此处不执行）
utils.print_html(full_content_round, title="以代码为计划（完整响应）")

# 执行函数（安全空间里执行LLM返回的python代码）

In [41]:
# --- 辅助函数：提取 <execute_python>...</execute_python> 之间的代码 ---
def _extract_execute_block(text: str) -> str:
    """
    返回 <execute_python>...</execute_python> 标签内的 Python 代码。
    如果未找到标签，则假定 'text' 已经是原始 Python 代码。
    """
    if not text:
        raise RuntimeError("空内容被传递给代码执行器。")
    m = re.search(r"<execute_python>(.*?)</execute_python>", text, re.DOTALL | re.IGNORECASE)
    return m.group(1).strip() if m else text.strip()


# ---------- 2) 代码执行 ----------
def execute_generated_code(
    code_or_content: str,
    *,
    db,
    inventory_tbl,
    transactions_tbl,
    user_request: Optional[str] = None,
) -> Dict[str, Any]:
    """
    在受控的命名空间中执行代码。
    接受原始 Python 代码 或 包含 <execute_python> 标签的完整内容。
    返回最小化的产物：stdout、error 和提取的答案。
    """
    # 在此处提取代码 (现在是集中处理)
    code = _extract_execute_block(code_or_content)

    SAFE_GLOBALS = {
        "Query": Query,
        "get_current_balance": inv_utils.get_current_balance,
        "next_transaction_id": inv_utils.next_transaction_id,
        "user_request": user_request or "",
    }
    SAFE_LOCALS = {
        "db": db,
        "inventory_tbl": inventory_tbl,
        "transactions_tbl": transactions_tbl,
    }

    # 捕获从被执行代码产生的 stdout
    _stdout_buf, _old_stdout = io.StringIO(), sys.stdout
    sys.stdout = _stdout_buf
    err_text = None
    try:
        exec(code, SAFE_GLOBALS, SAFE_LOCALS)
    except Exception:
        err_text = traceback.format_exc()
    finally:
        sys.stdout = _old_stdout
    printed = _stdout_buf.getvalue().strip()

    # 提取由生成代码设置的可能答案
    answer = (
        SAFE_LOCALS.get("answer_text")
        or SAFE_LOCALS.get("answer_rows")
        or SAFE_LOCALS.get("answer_json")
    )


    return {
        "code": code,           # <- 已经没有标签了
        "stdout": printed,
        "error": err_text,
        "answer": answer,
        "transactions_tbl": transactions_tbl.all(), # 供检查
        "inventory_tbl": inventory_tbl.all(),     # 供检查
    }

## 执行查询

In [42]:
# 针对圆形太阳镜问题执行生成的计划
result = execute_generated_code(
    full_content_round,          # 先前生成的完整 LLM 响应
    db=db,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    user_request=prompt_round,   # 例如："你们是否有库存中的圆形太阳镜，且价格低于 100 美元？"
)

# 查看计划实际执行的 Python 所提取的答案
utils.print_html(result["answer"], title="计划执行 · 提取的答案")

# 退货（看下库存表变动）

## 人工查询退货商品（库存表）

In [43]:
Item = Query()                    # 创建 Query 对象以引用字段（例如 Item.name、Item.description）

# 查询：提取所有 'name' 恰为 "Aviator" 的库存行。
# 说明：
# - 这是区分大小写的相等判断。"aviator" 不会匹配。
# - 若需不区分大小写匹配，可考虑 .test(...) 或 .matches(..., re.I)。
aviators = inventory_tbl.search(
    (Item.name == "Aviator")
)

# 以可读的 JSON 面板展示匹配文档
utils.print_html(json.dumps(aviators, indent=2), title="库存状态：退货前的 Aviator 太阳镜")

In [44]:
prompt_aviator = "退回我上周购买的 2 副 Aviator 太阳镜。"

# 生成以代码为计划的响应（完整内容；可能包含 <execute_python> 标签）
full_content_aviator = generate_llm_code(
    prompt_aviator,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    model="deepseek-chat",
    temperature=1,
)

# 查看 LLM 的计划与代码（此处不执行）
utils.print_html(full_content_aviator, title="以代码为计划（完整响应）")

## 查询交易表

In [45]:
utils.print_html(json.dumps(transactions_tbl.all(), indent=2), title="退货前的交易表")

## 执行退货后

In [46]:
# 执行针对 Aviator 太阳镜退货的生成计划
result = execute_generated_code(
    full_content_aviator,          # 先前生成的完整 LLM 响应
    db=db,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    user_request=prompt_aviator,   # 例如："退回我上周购买的 2 副 Aviator 太阳镜。"
)

# 查看计划实际执行的 Python 所提取的答案
utils.print_html(result["answer"], title="计划执行 · 提取的答案")

## 退货后交易表

In [47]:
utils.print_html(json.dumps(transactions_tbl.all(), indent=2), title="退货后的交易表")

## 退货后人工查询库存表

In [48]:
Item = Query()

aviators = inventory_tbl.search(
    (Item.name == "Aviator")
)

utils.print_html(json.dumps(aviators, indent=2), title="库存状态：退货后的 Aviator 太阳镜")
#数量从23变成25

# 客服智能体

**接受用户自然语言请求，LLM实现代码，在安全空间里执行交易（购买、退货、查询）**

In [49]:
def customer_service_agent(
    question: str,
    *,
    db,
    inventory_tbl,
    transactions_tbl,
    model: str = "o4-mini",
    temperature: float = 1.0,
    reseed: bool = False,
) -> dict:
    """
    端到端助手：
      1) (可选) 重新填充库存和交易数据
      2) 根据 `question` 生成“代码即计划”
      3) 在受控的命名空间中执行
      4) 渲染执行前后的快照并返回产物

    Returns:
      {
        "full_content": <原始 LLM 响应 (可能包含 <execute_python> 标签)>,
        "exec": {
            "code": <提取出的 python 代码>,
            "stdout": <计划日志>,
            "error": <回溯信息或 None>,
            "answer": <答案文本/行/json>,
            "inventory_after": [...],
            "transactions_after": [...]
        }
      }
    """
    # 0) 可选的重新填充数据
    if reseed:
        inv_utils.create_inventory()
        inv_utils.create_transactions()

    # 1) 显示问题
    utils.print_html(question, title="用户问题")

    # 2) 生成“代码即计划”(完整内容)
    full_content = generate_llm_code(
        question,
        inventory_tbl=inventory_tbl,
        transactions_tbl=transactions_tbl,
        model=model,
        temperature=temperature,
    )
    utils.print_html(full_content, title="以代码为计划（完整响应）")

    # 3) 执行前的快照
    utils.print_html(json.dumps(inventory_tbl.all(), indent=2), title="库存表 · 之前")
    utils.print_html(json.dumps(transactions_tbl.all(), indent=2), title="交易表 · 之前")

    # 4) 执行
    exec_res = execute_generated_code(
        full_content,
        db=db,
        inventory_tbl=inventory_tbl,
        transactions_tbl=transactions_tbl,
        user_request=question,
    )

    # 5) 执行后的快照 + 最终答案
    utils.print_html(exec_res["answer"], title="计划执行 · 提取的答案")
    utils.print_html(json.dumps(inventory_tbl.all(), indent=2), title="库存表 · 之后")
    utils.print_html(json.dumps(transactions_tbl.all(), indent=2), title="交易表 · 之后")

    # 6) 返回产物
    return {
        "full_content": full_content,
        "exec": {
            "code": exec_res["code"],
            "stdout": exec_res["stdout"],
            "error": exec_res["error"],
            "answer": exec_res["answer"],
            "inventory_after": inventory_tbl.all(),
            "transactions_after": transactions_tbl.all(),
        },
    }

reseed=True,每次请求前重置库存表（商品数量不变初始化）、交易表（余额恢复初始化），便于测试单次请求

In [50]:
prompt = "我想购买 3 副 Classic 太阳镜和 1 副 Aviator 太阳镜。"

out = customer_service_agent(
    prompt,
    db=db,
    inventory_tbl=inventory_tbl,
    transactions_tbl=transactions_tbl,
    model="deepseek-chat",
    temperature=1.0,
    reseed=True,   # 设为 False 以保留当前库存与交易状态
)